In [ ]:
from qcodes import Instrument


class ScaleSRS(Instrument):
    """
    This is only a demo!
    Feed me a SRS 830 lockin and I will automatically adjust the sensitivity on X measurements.
    """
    def __init__(self, name: str, srs, **kwargs) -> None:
        super().__init__(name, **kwargs)
        self.srs = srs
        self.sens = srs.sensitivity.get()
        self.add_parameter('X', get_cmd=self._getX)
        self.add_parameter('P', get_cmd=self.srs.P.get)

        
    def _getX(self):
        x = self.srs.X.get()
        dn = 0
        if x > self.sens * 0.9:
            dn = 1
        elif x < self.sens * 0.1:
            dn = -1
        else:
            return x
        n = self.srs._VOLT_TO_N[self.sens]
        if n + dn > 26 or n + dn < 0:
            return x
        new = self.srs._N_TO_VOLT[n + dn]
        self.srs.sensitivity.set(new)
        self.sens = new
        time.sleep(3)
        return self.srs.X.get()

In [ ]:
from qcodes.instrument_drivers.stanford_research.SR830 import SR830
srs = SR830('srs', 'GPIB1::10::INSTR')
srs_autoscaled = ScaleSRS('srs_autoscaled', srs)
srs_autoscaled.X.get()
srs_autoscaled.P.get()